In [ ]:
"""
Generate surface variables from DEM for informative value procedure
"""

mdt        = '/mnt/d/mystuff/pwgis/mod5/mdt.tif'
rules      = '/mnt/d/mystuff/pwgis/mod5/rules.json'

datafolder = '/mnt/d/mystuff/pwgis/mod5/datavar'
varfolder  = '/mnt/d/mystuff/pwgis/mod5/var'

In [ ]:
import os
import json as js
from glass.pys.oss    import fprop
from glass.wenv.grs import run_grass
from glass.prop.prj import get_epsg
from glass.rst.rcls import interval_rules
from glass.prop.rst import rst_stats
from glass.rst.surf.wtr import flow_accum
from glass.rst.alg import grsrstcalc
from glass.rst.surf import curvature

In [ ]:
# Reclassify rules

dict_rules = js.load(open(rules, 'r'))

# Create reclassify files
for k in dict_rules:
    dict_rules[k] = interval_rules(
        {_k : [
            v * 10000 for v in dict_rules[k][_k]
        ] for _k in dict_rules[k]},
        os.path.join(datafolder, k.lower() + '.txt')
    )

In [ ]:
# Get name for GRASS GIS location
loc = fprop(mdt, 'fn', forceLower=True)[:5] + '_loc'

# Get DEM EPSG

epsg = get_epsg(mdt)

# Create GRASS GIS location
gbase = run_grass(datafolder, location=loc, srs=epsg)

# Start GRASS GIS Session
import grass.script.setup as gsetup
gsetup.init(gbase, datafolder, loc, 'PERMANENT')

In [ ]:
# Import GRASS GIS modules
from glass.it.rst   import rst_to_grs, grs_to_rst
from glass.wenv.grs import rst_to_region
from glass.rst.surf import slope, aspect
from glass.rst.rcls import interval_rules, rcls_rst

In [ ]:
# Import MDT
grsmdt = rst_to_grs(mdt, 'mdt')

# Set Region
rst_to_region(grsmdt)

# Generate slope
grsslope = slope(grsmdt, 'sloperst', data='degrees', api="grass")

# Generate Aspect
grsexpo = aspect(grsmdt, 'aspectrst', from_north=True, api="grass")

In [ ]:
# Reclassify elevation
mdt_int = grsrstcalc(f"int({grsmdt} * 10000)", "mdtint")
var_elevation = rcls_rst(mdt_int, dict_rules["ELEVATION"], 'elevation', api="pygrass")

# Reclassify slope
slope_int = grsrstcalc(f"int({grsslope} * 10000)", "slopeint")
var_slope = rcls_rst(slope_int, dict_rules["SLOPE"], 'slope', api="pygrass")

# Reclassify aspect
expo_int = grsrstcalc(f"int({grsexpo} * 10000)", "expoint")
var_aspect = rcls_rst(expo_int, dict_rules["ASPECT"], 'aspect', api="pygrass")

In [ ]:
# Generate Inverse Topographic Index

# Get flow direction
flowrst, flowdir = flow_accum(grsmdt, 'rst_accum', 'rst_dir', ascmd=True)

# Calculate ITI
topoidx = grsrstcalc(f"{grsslope} / {flowrst}", "invtopoidx")
topoidxint = grsrstcalc(f"int({topoidx} * 10000)", "topoidxint")

# Reclassify
# Menores valores = mais humidade no solo
invtoporules = interval_rules({
    1 : [-1 * 10000, 0],
    2 : [0, int(0.0001 * 10000)],
    3 : [int(0.0001 * 10000), int(0.001 * 10000)],
    4 : [int(0.001 * 10000), int(0.01 * 10000)],
    5 : [int(0.01 * 10000), int(0.1 * 10000)],
    6 : [int(0.1 * 10000), int(100 * 10000)]
}, os.path.join(datafolder, 'invtoporules.txt'))

varinvtopo = rcls_rst(topoidxint, invtoporules, 'varinvtopoidx', api='pygrass')

In [ ]:
# Generate curvature rasters

pcurv, tcurv = curvature(grsmdt, 'pcurv', 'tcurv', ascmd=True)

pcurvint = grsrstcalc(f"int({pcurv} * 10000)", "pcurvint")

# Reclassify profile curvature

curvrules = interval_rules({
    1 : [-2 * 10000, int(-0.025 * 10000)],
    2 : [int(-0.025 * 10000), int(-0.0025 * 10000)],
    3 : [int(-0.0025 * 10000), int(0.0025 * 10000)],
    4 : [int(0.0025 * 10000), int(0.025 * 10000)],
    5 : [int(0.025 * 10000), int(2 * 10000)]
}, os.path.join(datafolder, 'varcurvrules.txt'))
varcurv = rcls_rst(pcurvint, curvrules, 'varcurv', api='pygrass')

In [ ]:
# Export data
drst = [
    grsslope, grsexpo, flowrst, flowdir, topoidx, pcurv,
    mdt_int, slope_int, expo_int, topoidxint, pcurvint
]
vrst = [var_elevation, var_slope, var_aspect, varinvtopo, varcurv]

for r in drst:
    grs_to_rst(r, os.path.join(datafolder, f'{r}.tif'), as_cmd=True)
for r in vrst:
    grs_to_rst(r, os.path.join(varfolder, f"{r}.tif"), as_cmd=True)